In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog, local_binary_pattern

DATASET_PATH = "dataset" 
CATEGORIES = ["cars", "Bike"] # Label 0 = cars, Label 1 = motorcycles
OUTPUT_CSV = "features_mobil_motor.csv"

# --- 1. FUNGSI: Ekstraksi Warna (Color Histogram) ---
def extract_color_histogram(image, bins=32):
    # Menghitung dominasi warna (Merah/Hijau/Biru)
    b_hist = cv2.calcHist([image], [0], None, [bins], [0, 256])
    g_hist = cv2.calcHist([image], [1], None, [bins], [0, 256])
    r_hist = cv2.calcHist([image], [2], None, [bins], [0, 256])
    
    # Normalisasi agar nilainya seimbang
    cv2.normalize(b_hist, b_hist)
    cv2.normalize(g_hist, g_hist)
    cv2.normalize(r_hist, r_hist)
    
    return np.concatenate([b_hist, g_hist, r_hist]).flatten()

# --- 2. FUNGSI: Ekstraksi Tekstur (LBP) ---
def extract_texture_lbp(gray_image, num_points=24, radius=8):
    # LBP melihat pola kasar/halus permukaan
    lbp = local_binary_pattern(gray_image, num_points, radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), 
                             bins=np.arange(0, num_points + 3), 
                             range=(0, num_points + 2))
    
    # Normalisasi
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return hist

# --- 3. FUNGSI: Ekstraksi Bentuk (HOG) ---
def extract_shape_hog(gray_image):
    # HOG melihat garis-garis bentuk body mobil/motor
    # Resize ke 64x64 agar jumlah fitur konsisten
    resized_img = cv2.resize(gray_image, (64, 64)) 
    
    fd, _ = hog(resized_img, 
                orientations=9, 
                pixels_per_cell=(8, 8), 
                cells_per_block=(2, 2), 
                visualize=True)
    return fd

# --- 4. FUNGSI: Ekstraksi Lokal (ORB) ---
def extract_local_orb(gray_image):
    # ORB melihat titik unik (misal: logo, lampu, jari-jari roda)
    orb = cv2.ORB_create(nfeatures=10)
    keypoints, descriptors = orb.detectAndCompute(gray_image, None)
    
    if descriptors is None:
        return np.zeros(32) # Jika gagal deteksi, kasih nilai 0
    
    # Ambil rata-rata agar ukurannya tetap
    return np.mean(descriptors, axis=0)

# --- MAIN LOOP (Jantung Program) ---
data_features = []
print("Mulai proses ekstraksi fitur...")

for label_id, category in enumerate(CATEGORIES):
    folder_path = os.path.join(DATASET_PATH, category)
    print(f"\nSedang memproses folder: {category} (Label: {label_id})")
    
    # Cek apakah folder ada
    if not os.path.exists(folder_path):
        print(f"ERROR: Folder '{folder_path}' tidak ditemukan! Cek nama foldermu.")
        continue

    count = 0
    for filename in os.listdir(folder_path):
        # Hanya baca file gambar
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(folder_path, filename)
            
            # Baca Gambar
            image = cv2.imread(img_path)
            if image is None: continue
            
            # Konversi ke Grayscale (Hitam Putih) untuk fitur tekstur & bentuk
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            try:
                # --- EKSTRAKSI SEMUA FITUR ---
                f_color = extract_color_histogram(image)   # Warna
                f_textur = extract_texture_lbp(gray)       # Tekstur
                f_shape = extract_shape_hog(gray)          # Bentuk
                f_local = extract_local_orb(gray)          # Lokal
                
                # --- GABUNGKAN FITUR + LABEL ---
                # Urutan: [nama_file, fitur_1, fitur_2, ..., fitur_ribuan, LABEL]
                global_feature = np.hstack([f_color, f_textur, f_shape, f_local])
                data_features.append([filename] + global_feature.tolist() + [label_id])
                
                count += 1
                if count % 20 == 0: print(f"  - Berhasil ekstrak {count} gambar...")
                
            except Exception as e:
                print(f"  - Gagal memproses {filename}: {e}")

# --- SIMPAN KE CSV ---
if data_features:
    # Buat nama kolom otomatis
    # Kolom terakhir kita namai 'label'
    num_cols = len(data_features[0]) - 2 # Dikurangi filename & label
    cols = ['filename'] + [f'feat_{j}' for j in range(num_cols)] + ['label']
    
    df = pd.DataFrame(data_features, columns=cols)
    
    # Simpan!
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\nSUKSES BESAR! 🎉")
    print(f"File '{OUTPUT_CSV}' berhasil dibuat.")
    print(f"Total Data: {len(df)} baris")
    print(f"Jumlah Fitur per Gambar: {num_cols}")
    print("Sekarang lanjut jalankan codingan Training ANN (Cell 1-6) yang tadi!")
else:
    print("\nGAGAL: Tidak ada data yang tersimpan. Cek folder 'dataset' kamu.")

Mulai proses ekstraksi fitur...

Sedang memproses folder: cars (Label: 0)
  - Berhasil ekstrak 20 gambar...
  - Berhasil ekstrak 40 gambar...
  - Berhasil ekstrak 60 gambar...
  - Berhasil ekstrak 80 gambar...
  - Berhasil ekstrak 100 gambar...

Sedang memproses folder: Bike (Label: 1)
  - Berhasil ekstrak 20 gambar...
  - Berhasil ekstrak 40 gambar...
  - Berhasil ekstrak 60 gambar...
  - Berhasil ekstrak 80 gambar...
  - Berhasil ekstrak 100 gambar...

SUKSES BESAR! 🎉
File 'features_mobil_motor.csv' berhasil dibuat.
Total Data: 200 baris
Jumlah Fitur per Gambar: 1918
Sekarang lanjut jalankan codingan Training ANN (Cell 1-6) yang tadi!
